In [ ]:
import os
import glob
import numpy as np
import pandas as pd

get sex from metadata and create lists

In [ ]:
metadata = "/mnt/sda1/Repos/a-eye/Output/metadata/sub_metadata.csv"
metadata_df = pd.read_csv(metadata)

males = metadata_df[metadata_df['Sex'] == 'M']
male_list = males['Subject'].tolist() # 594

females = metadata_df[metadata_df['Sex'] == 'F']
female_list = females['Subject'].tolist() # 616

create input folders

In [ ]:
import shutil

# Define the source directory where the files are located
source_directory = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_cropped'

# Define the destination directory where the folder will be created
destination_directory = '/home/jaimebarranco/Desktop/eye_model/female/input'

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# Iterate over the subjects in the list
for subject in female_list:
    # Generate the source file path based on the subject ID
    source_file = os.path.join(source_directory, f'{subject}_cropped.nii.gz')
    
    # Check if the source file exists
    if os.path.exists(source_file):        
        # Copy the source file to the destination folder
        shutil.copy(source_file, destination_directory)

template construction

In [ ]:
os.environ['PATH'] += ':/opt/ANTs/bin' # ensure the ANTs binaries are in $PATH
os.environ['ANTSPATH'] = '/opt/ANTs/bin/'

In [ ]:
input_dir = '/home/jaimebarranco/Desktop/eye_model/combined/input/'
output_dir = '/home/jaimebarranco/Desktop/eye_model/combined/output_template_construction/'

# Create output directories
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(input_dir):
    os.makedirs(input_dir)

# v2
command = 'antsMultivariateTemplateConstruction2.sh -d 3' + \
    ' -o ' + output_dir + \
    ' -i ' + '15' + \
    ' -g ' + '0.2' + \
    ' -j ' + '16' + \
    ' -c ' + '2' + \
    ' -k ' + '1' + \
    ' -w ' + '1' + \
    ' -n ' + '1' + \
    ' -r ' + '1' + \
    ' -m ' + 'CC[2]' + \
    ' -q ' + '80x60x40x10' + \
    ' -f ' + '8x4x2x1' + \
    ' -s ' + '3x2x1x0' + \
    ' -t ' + 'SyN' + \
    ' ' + input_dir + '*.nii.gz'
print(command)
# os.system(command)

crop volume

In [ ]:
import nibabel as nb
import SimpleITK as sitk
import pandas as pd

template = '/home/jaimebarranco/Desktop/eye_model/combined/output_template_construction/template0.nii.gz'

template_sitk = sitk.ReadImage(template)
template_nb = nb.load(template)

In [ ]:
# Get the data from the nibabel image
data = template_nb.get_fdata()

# Find the indices where the image is not zero
indices = np.where(data != 0)

# Get the minimum and maximum coordinates for each dimension
min_coords = np.min(indices, axis=1)
max_coords = np.max(indices, axis=1)

# The bounding box is then given by the minimum and maximum coordinates
bounding_box = np.vstack((min_coords, max_coords)).T

print("Bounding box (in voxel coordinates):")
print(bounding_box)